In [1]:
import math
import re
from random import *
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from sklearn.model_selection import train_test_split
import ast
import itertools
import os

os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

device = torch.device("cuda")

In [2]:
df = pd.read_csv("./out/demo_labelled_dataset.csv")

df

,a[0],a[1],a[2],a[3],eigen_values,results,numeric_output
0,9,-2,2,-3,"['8.65685424949238', '-2.65685424949238']",1,"([ 1.00000000e+00, -1.20000000e+01, 9.9977247..."
1,-9,-3,2,-4,"['-7.0', '-6.0']",0,"([ 1.00000000e+00, -1.40000000e+01, 9.9971614..."
2,-5,-9,2,1,"['(-1.9999999999999998+3.000000000000001j)', '...",0,"([ 1.00000000e+00, -1.40000000e+01, 9.9973320..."
3,-7,-7,-1,-5,"['-8.82842712474619', '-3.17157287525381']",0,"([0], [0])"
4,7,-7,-9,4,"['13.577747210701755', '-2.577747210701756']",1,"([ 0, 2, 4, 6, 8, 10, 12, ..."
...,...,...,...,...,...,...,...
996,2,-6,8,-9,"['(-3.500000000000001+4.21307488658818j)', '(-...",0,NaN
997,8,-9,4,4,"['(6+5.65685424949238j)', '(6-5.65685424949238...",1,NaN
998,-3,-9,8,-9,"['(-6+7.937253933193773j)', '(-6-7.93725393319...",0,NaN
999,-6,-1,4,-7,"['(-6.5+1.9364916731037083j)', '(-6.5-1.936491...",0,NaN


In [3]:
X = df["numeric_output"][:926]
y = df.results[:926]

X, y

(0      ([ 1.00000000e+00, -1.20000000e+01,  9.9977247...
 1      ([ 1.00000000e+00, -1.40000000e+01,  9.9971614...
 2      ([ 1.00000000e+00, -1.40000000e+01,  9.9973320...
 3                                             ([0], [0])
 4      ([   0,    2,    4,    6,    8,   10,   12,   ...
                              ...                        
 921    ([ 1.00000000e+00, -1.70000000e+01,  9.9971169...
 922    ([1.00000000e+00, 6.00000000e+00, 1.00015154e+...
 923                                           ([0], [0])
 924    ([   0,    2,    4,    6,    8,   10,   12,   ...
 925                                           ([0], [0])
 Name: numeric_output, Length: 926, dtype: object,
 0      1
 1      0
 2      0
 3      0
 4      1
       ..
 921    1
 922    1
 923    0
 924    1
 925    1
 Name: results, Length: 926, dtype: int64)

In [4]:
X = X.apply(ast.literal_eval)

print(type(X))

<class 'pandas.core.series.Series'>


In [5]:
train_X, val_X, train_y, val_y = train_test_split(X, y)

train_X, val_X 

(719    ([0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 2...
 198    ([0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 2...
 422    ([0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 2...
 199    ([1.0, -8.0, 0.999807164, -7.99843321, 0.99961...
 595    ([1.0, 11.0, 1.00024492, 10.999488, 1.00048982...
                              ...                        
 176                                           ([0], [0])
 107                                           ([0], [0])
 757                                           ([0], [0])
 791    ([0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 2...
 158    ([0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 2...
 Name: numeric_output, Length: 694, dtype: object,
 845                                           ([0], [0])
 443    ([1.0, -2.0, 0.999943579, -1.99943575, 0.99988...
 740                                           ([0], [0])
 828    ([1.0, -9.0, 0.999808936, -8.9984715, 0.999617...
 753    ([1.0, 0.0, 0.999999966, -0.000321473102, 0.99...
                     

In [6]:
# numeric_series = X

numeric_series = pd.Series([
    ([1.0], [-12.0, 0.999772471]),
    ([-11.9978384],[ 0.999544983]),
    ([0.99909009], [-11.9913555, 0.99863536]),
    ([-11.9870352],[ 0.998180794]),
    ([0.993644133], [-11.9396045, 0.989123796]),
    ([-11.9913555, 0.99863536], [-12.0, 0.999772471]),
    ([-11.9396045],[ 0.999544983]),
    ([0.99909009], [-11.9870352]),
    ([-11.9870352],[ 0.998180794]),
    ([0.993644133], [-11.9396045, 0.989123796])
])

word_dict = {'[PAD]': 0, '[CLS]': 1, '[SEP]': 2, '[MASK]': 3}
vocab_size = len(word_dict)

for lst1, lst2 in numeric_series:
    for i, num in enumerate(lst1):
        token = str(num)
        if token not in word_dict:
            word_dict[token] = vocab_size
            vocab_size += 1
    for i, num in enumerate(lst2):
        token = str(num)
        if token not in word_dict:
            word_dict[token] = vocab_size
            vocab_size += 1

number_dict = {i: w for w, i in word_dict.items()}

tokenized_series = numeric_series.apply(lambda lst: [word_dict.get(str(num), num) for num in lst])

print(tokenized_series)

token_list = list()
for xsol, ysol in tokenized_series:
    x_tokens = []
    y_tokens = []
    for num in xsol:
        x_tokens.append(word_dict[str(num)])
    for num in ysol:
        y_tokens.append(word_dict[str(num)])
    tokens = [x_tokens, y_tokens]
    token_list.append(tokens)

0                        [[1.0], [-12.0, 0.999772471]]
1                       [[-11.9978384], [0.999544983]]
2            [[0.99909009], [-11.9913555, 0.99863536]]
3                       [[-11.9870352], [0.998180794]]
4          [[0.993644133], [-11.9396045, 0.989123796]]
5    [[-11.9913555, 0.99863536], [-12.0, 0.999772471]]
6                       [[-11.9396045], [0.999544983]]
7                        [[0.99909009], [-11.9870352]]
8                       [[-11.9870352], [0.998180794]]
9          [[0.993644133], [-11.9396045, 0.989123796]]
dtype: object


In [7]:
# merge tokens to a single list
# token_list = list(word_dict.values())
word_dict

{'[PAD]': 0,
 '[CLS]': 1,
 '[SEP]': 2,
 '[MASK]': 3,
 '1.0': 4,
 '-12.0': 5,
 '0.999772471': 6,
 '-11.9978384': 7,
 '0.999544983': 8,
 '0.99909009': 9,
 '-11.9913555': 10,
 '0.99863536': 11,
 '-11.9870352': 12,
 '0.998180794': 13,
 '0.993644133': 14,
 '-11.9396045': 15,
 '0.989123796': 16}

In [8]:
token_list

[[[4], [5, 6]],
 [[7], [8]],
 [[9], [10, 11]],
 [[12], [13]],
 [[14], [15, 16]],
 [[10, 11], [5, 6]],
 [[15], [8]],
 [[9], [12]],
 [[12], [13]],
 [[14], [15, 16]]]

In [9]:
# text = (
#         'Hello, how are you? I am Romeo.\n'
#         'Hello, Romeo My name is Juliet. Nice to meet you.\n'
#         'Nice meet you too. How are you today?\n'
#         'Great. My baseball team won the competition.\n'
#         'Oh Congratulations, Juliet\n'
#         'Thank you Romeo'
#     )

In [10]:
# sentences = re.sub("[.,!?\\-]", '', text.lower()).split('\n')
# word_list = list(set(" ".join(sentences).split()))
# word_dict = {'[PAD]': 0, '[CLS]': 1, '[SEP]': 2, '[MASK]': 3}


# for i, w in enumerate(word_list):
#     word_dict[w] = i + 4
# number_dict = {i: w for i, w in enumerate(word_dict)}
# vocab_size = len(word_dict)

# token_list = list()
# for sentence in sentences:
#     arr = [word_dict[s] for s in sentence.split()]
#     token_list.append(arr)

In [11]:
# token_list

In [12]:
# maxlen = 30 
# batch_size = 6
# max_pred = 5  
# n_layers = 6 
# n_heads = 12 
# d_model = 768 
# d_ff = 768 * 4  
# d_k = d_v = 64  
# n_segments = 2

In [13]:
maxlen = 30 
batch_size = 6
max_pred = 5  
n_layers = 6 
n_heads = 12 
d_model = 32 
d_ff = 32 * 4
d_k = d_v = 2
n_segments = 2

In [14]:
def make_batch():
    batch = []
    positive = negative = 0
    while positive != batch_size/2 or negative != batch_size/2:
        tokens_a_index, tokens_b_index= randrange(len(tokenized_series)), randrange(len(tokenized_series))
        tokens_a, tokens_b= token_list[tokens_a_index], token_list[tokens_b_index]

        input_ids = [word_dict['[CLS]']] + tokens_a + [word_dict['[SEP]']] + tokens_b + [word_dict['[SEP]']]

        segment_ids = [0] * (1 + len(tokens_a) + 1) + [1] * (len(tokens_b) + 1)

        n_pred =  min(max_pred, max(1, int(round(len(input_ids) * 0.15)))) 

        cand_maked_pos = [i for i, token in enumerate(input_ids)
                          if token != word_dict['[CLS]'] and token != word_dict['[SEP]']]
        shuffle(cand_maked_pos)
        masked_tokens, masked_pos = [], []
        for pos in cand_maked_pos[:n_pred]:
            masked_pos.append(pos)
            masked_tokens.append(input_ids[pos])
            if random() < 0.8:  
                input_ids[pos] = word_dict['[MASK]'] 
            elif random() < 0.5:  
                index = randint(0, vocab_size - 1) 
                input_ids[pos] = word_dict[number_dict[index]] 
        
        n_pad = maxlen - len(input_ids)
        input_ids.extend([0] * n_pad)
        segment_ids.extend([0] * n_pad)
    
        if max_pred > n_pred:
            n_pad = max_pred - n_pred
            masked_tokens.extend([0] * n_pad)
            masked_pos.extend([0] * n_pad)

        if tokens_a_index + 1 == tokens_b_index and positive < batch_size/2:
            batch.append([input_ids, segment_ids, masked_tokens, masked_pos, True]) 
            positive += 1
        elif tokens_a_index + 1 != tokens_b_index and negative < batch_size/2:
            batch.append([input_ids, segment_ids, masked_tokens, masked_pos, False]) 
            negative += 1
    return batch

In [15]:
# def make_batch():
#     batch = []
#     positive = negative = 0
#     while positive != batch_size/2 or negative != batch_size/2:
#         tokens_a_index, tokens_b_index= randrange(len(sentences)), randrange(len(sentences))
#         tokens_a, tokens_b= token_list[tokens_a_index], token_list[tokens_b_index]

#         input_ids = [word_dict['[CLS]']] + tokens_a + [word_dict['[SEP]']] + tokens_b + [word_dict['[SEP]']]

#         segment_ids = [0] * (1 + len(tokens_a) + 1) + [1] * (len(tokens_b) + 1)

#         n_pred =  min(max_pred, max(1, int(round(len(input_ids) * 0.15)))) 

#         cand_maked_pos = [i for i, token in enumerate(input_ids)
#                           if token != word_dict['[CLS]'] and token != word_dict['[SEP]']]
#         shuffle(cand_maked_pos)
#         masked_tokens, masked_pos = [], []
#         for pos in cand_maked_pos[:n_pred]:
#             masked_pos.append(pos)
#             masked_tokens.append(input_ids[pos])
#             if random() < 0.8:  
#                 input_ids[pos] = word_dict['[MASK]'] 
#             elif random() < 0.5:  
#                 index = randint(0, vocab_size - 1) 
#                 input_ids[pos] = word_dict[number_dict[index]] 
        
#         n_pad = maxlen - len(input_ids)
#         input_ids.extend([0] * n_pad)
#         segment_ids.extend([0] * n_pad)
    
#         if max_pred > n_pred:
#             n_pad = max_pred - n_pred
#             masked_tokens.extend([0] * n_pad)
#             masked_pos.extend([0] * n_pad)

#         if tokens_a_index + 1 == tokens_b_index and positive < batch_size/2:
#             batch.append([input_ids, segment_ids, masked_tokens, masked_pos, True]) 
#             positive += 1
#         elif tokens_a_index + 1 != tokens_b_index and negative < batch_size/2:
#             batch.append([input_ids, segment_ids, masked_tokens, masked_pos, False]) 
#             negative += 1
#     return batch

In [16]:
def get_attn_pad_mask(seq_q, seq_k):
    batch_size, len_q = seq_q.size()
    batch_size, len_k = seq_k.size()
    pad_attn_mask = seq_k.data.eq(0).unsqueeze(1)
    pad_attn_mask = pad_attn_mask.to(device)
    return pad_attn_mask.expand(batch_size, len_q, len_k)  

In [17]:
def gelu(x):
    return x * 0.5 * (1.0 + torch.erf(x / math.sqrt(2.0)))

In [18]:
batch = make_batch()
print("done", type(batch))

done <class 'list'>


In [19]:
def flatten_tokens(tokens):
    flattened_tokens = []
    for nums in tokens:
        if type(nums) is list:
            flattened_tokens.extend(nums)
        else:
            flattened_tokens.append(nums)
    
    return flattened_tokens

In [20]:
def arrange_tokens(tokens):
    arranged = []
    for nums in tokens:
        flattened = flatten_tokens(nums)
        length = len(flattened)
        if length != 32:
            missing = 32 - length
            flattened.extend([0] * missing)
        arranged.append(flattened)
    
    return arranged

In [21]:
# def flatten_tokens(nestedList):
# 	if not(bool(nestedList)):
# 		return nestedList

# 	if isinstance(nestedList[0], list):
# 		return flatten_tokens(*nestedList[:1]) + flatten_tokens(nestedList[1:])

# 	return nestedList[:1] + flatten_tokens(nestedList[1:])

In [22]:
# input_ids, segment_ids, masked_tokens, masked_pos, isNext = map(torch.LongTensor, zip(*batch))

# input_ids = input_ids.to(device)
# segment_ids = segment_ids.to(device)
# masked_tokens = masked_tokens.to(device)
# masked_pos = masked_pos.to(device)
# isNext = isNext.to(device)

In [23]:
# input_ids, segment_ids, masked_tokens, masked_pos, isNext = map(torch.LongTensor, zip(*batch))

input_ids = torch.LongTensor(arrange_tokens([nums[0] for nums in batch]))
segment_ids = torch.LongTensor([nums[1] for nums in batch])
masked_tokens = torch.LongTensor(arrange_tokens([nums[2] for nums in batch]))
masked_pos = torch.LongTensor([nums[3] for nums in batch])
isNext = torch.LongTensor([nums[4] for nums in batch])

input_ids = input_ids.to(device)
segment_ids = segment_ids.to(device)
masked_tokens = masked_tokens.to(device)
masked_pos = masked_pos.to(device)
isNext = isNext.to(device)

In [24]:
get_attn_pad_mask(input_ids, input_ids)[0][0], input_ids[0], segment_ids[0], masked_tokens[0], masked_pos[0], isNext[0]

(tensor([False, False, False, False, False, False, False,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True], device='cuda:0'),
 tensor([ 1, 15,  8,  2, 14,  3,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        device='cuda:0'),
 tensor([0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0], device='cuda:0'),
 tensor([15, 16,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        device='cuda:0'),
 tensor([5, 0, 0, 0, 0], device='cuda:0'),
 tensor(0, device='cuda:0'))

In [25]:
class Embedding(nn.Module):
    def __init__(self):
        super(Embedding, self).__init__()
        self.tok_embed = nn.Embedding(vocab_size, d_model)
        self.pos_embed = nn.Embedding(maxlen, d_model)
        self.seg_embed = nn.Embedding(n_segments, d_model)
        self.norm = nn.LayerNorm(d_model)

    def forward(self, x, seg):
        seq_len = x.size(1)
        pos = torch.arange(seq_len, dtype=torch.long)
        pos = pos.unsqueeze(0).expand_as(x)
        x = x.to(device)
        pos = pos.to(device)
        seg = seg.to(device)
        embedding = self.tok_embed(x) + self.pos_embed(pos) + self.seg_embed(seg)
        embedding = embedding.to(device)
        return self.norm(embedding)

In [26]:
class ScaledDotProductAttention(nn.Module):
    def __init__(self):
        super(ScaledDotProductAttention, self).__init__()

    def forward(self, Q, K, V, attn_mask):
        scores = torch.matmul(Q, K.transpose(-1, -2)) / np.sqrt(d_k)
        scores.masked_fill_(attn_mask, -1e9)
        scores = scores.to(device)
        attn = nn.Softmax(dim=-1)(scores)
        attn = attn.to(device)
        context = torch.matmul(attn, V)
        context = context.to(device)
        return scores, context, attn 

In [27]:
emb = Embedding()
emb = emb.to(device)
input_ids = input_ids.to(device)
segment_ids = segment_ids.to(device)
embeds = emb(input_ids, segment_ids)

attenM = get_attn_pad_mask(input_ids, input_ids)

attenM = attenM.to(device)

SDPA= ScaledDotProductAttention()(embeds, embeds, embeds, attenM)

S, C, A = SDPA

S = S.to(device)
C = C.to(device)
A = A.to(device)


print('Masks',attenM[0][0])
print()
print('Scores: ', S[0][0],'\n\nAttention M: ', A[0][0])

RuntimeError: CUDA error: device-side assert triggered
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
class MultiHeadAttention(nn.Module):
    def __init__(self):
        super(MultiHeadAttention, self).__init__()
        self.W_Q = nn.Linear(d_model, d_k * n_heads)
        self.W_K = nn.Linear(d_model, d_k * n_heads)
        self.W_V = nn.Linear(d_model, d_v * n_heads)
    def forward(self, Q, K, V, attn_mask):
        Q = Q.to(device)
        K = K.to(device)
        V = V.to(device)
        attn_mask = attn_mask.to(device)
        
        residual, batch_size = Q, Q.size(0)
        q_s = self.W_Q(Q).view(batch_size, -1, n_heads, d_k).transpose(1,2)
        k_s = self.W_K(K).view(batch_size, -1, n_heads, d_k).transpose(1,2)
        v_s = self.W_V(V).view(batch_size, -1, n_heads, d_v).transpose(1,2)

        attn_mask = attn_mask.unsqueeze(1).repeat(1, n_heads, 1, 1)
        
        _, context, attn = ScaledDotProductAttention().to(device)(q_s, k_s, v_s, attn_mask)
        attn = attn.to(device)
        context = context.transpose(1, 2).contiguous().view(batch_size, -1, n_heads * d_v)
        context = context.to(device)
        output = nn.Linear(n_heads * d_v, d_model).to(device)(context)
        return nn.LayerNorm(d_model).to(device)(output + residual), attn


In [ ]:
emb = Embedding()
emb = emb.to(device)
input_ids = input_ids.to(device)
segment_ids = segment_ids.to(device)
embeds = emb(input_ids, segment_ids)
embeds = embeds.to(device)

attenM = get_attn_pad_mask(input_ids, input_ids)
attenM = attenM.to(device)

MHA= MultiHeadAttention().to(device)(embeds, embeds, embeds, attenM)

Output, A = MHA
Output = Output.to(device)
A = A.to(device)

A[0][0]

In [ ]:
class PoswiseFeedForwardNet(nn.Module):
    def __init__(self):
        super(PoswiseFeedForwardNet, self).__init__()
        self.fc1 = nn.Linear(d_model, d_ff)
        self.fc2 = nn.Linear(d_ff, d_model)

    def forward(self, x):
        return self.fc2(gelu(self.fc1(x)))


In [ ]:
class EncoderLayer(nn.Module):
    def __init__(self):
        super(EncoderLayer, self).__init__()
        self.enc_self_attn = MultiHeadAttention()
        self.pos_ffn = PoswiseFeedForwardNet()

    def forward(self, enc_inputs, enc_self_attn_mask):
        enc_outputs, attn = self.enc_self_attn(enc_inputs, enc_inputs, enc_inputs, enc_self_attn_mask)
        attn = attn.to(device)
        enc_outputs = self.pos_ffn(enc_outputs)
        enc_outputs = enc_outputs.to(device)
        return enc_outputs, attn

In [ ]:
class BERT(nn.Module):
    def __init__(self):
        super(BERT, self).__init__()
        self.embedding = Embedding()
        self.embedding = self.embedding.to(device)
        self.layers = nn.ModuleList([EncoderLayer() for _ in range(n_layers)])
        self.fc = nn.Linear(d_model, d_model)
        self.activ1 = nn.Tanh()
        self.linear = nn.Linear(d_model, d_model)
        self.activ2 = gelu
        self.norm = nn.LayerNorm(d_model)
        self.classifier = nn.Linear(d_model, 2)
        embed_weight = self.embedding.tok_embed.weight
        embed_weight.to(device)
        n_vocab, n_dim = embed_weight.size()
        self.decoder = nn.Linear(n_dim, n_vocab, bias=False)
        self.decoder.weight = embed_weight
        self.decoder_bias = nn.Parameter(torch.zeros(n_vocab))

    def forward(self, input_ids, segment_ids, masked_pos):
        input_ids = input_ids.to(device)
        segment_ids = segment_ids.to(device)
        masked_pos = masked_pos.to(device)

        output = self.embedding(input_ids, segment_ids)
        enc_self_attn_mask = get_attn_pad_mask(input_ids, input_ids).to(device)
        for layer in self.layers:
            output, enc_self_attn = layer(output, enc_self_attn_mask)
        h_pooled = self.activ1(self.fc(output[:, 0])) 
        logits_clsf = self.classifier(h_pooled) 

        masked_pos = masked_pos[:, :, None].expand(-1, -1, output.size(-1)) 
        
        h_masked = torch.gather(output, 1, masked_pos) 
        h_masked = self.norm(self.activ2(self.linear(h_masked)))
        logits_lm = self.decoder(h_masked) + self.decoder_bias 

        return logits_lm, logits_clsf

In [ ]:
model = BERT()

model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

batch = make_batch()
input_ids, segment_ids, masked_tokens, masked_pos, isNext = map(torch.LongTensor, zip(*batch))
input_ids = input_ids.to(device)
segment_ids = segment_ids.to(device)
masked_tokens = masked_tokens.to(device)
masked_pos = masked_pos.to(device)
isNext = isNext.to(device)

for epoch in range(10):
    optimizer.zero_grad()
    logits_lm, logits_clsf = model(input_ids.to(device), segment_ids.to(device), masked_pos.to(device))
    loss_lm = criterion(logits_lm.transpose(1, 2), masked_tokens) 
    loss_lm = (loss_lm.float()).mean()
    loss_clsf = criterion(logits_clsf, isNext) 
    loss = loss_lm + loss_clsf
    if (epoch + 1) % 10 == 0:
        print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.6f}'.format(loss))
    loss.backward()
    optimizer.step()

In [ ]:
# input_ids, segment_ids, masked_tokens, masked_pos, isNext = map(torch.LongTensor, zip(batch[0]))
# print(text)
# print([number_dict[w.item()] for w in input_ids[0] if number_dict[w.item()] != '[PAD]'])

# input_ids = input_ids.to(device)
# segment_ids = segment_ids.to(device)
# masked_tokens = masked_tokens.to(device)
# masked_pos = masked_pos.to(device)
# isNext = isNext.to(device)

# logits_lm, logits_clsf = model(input_ids, segment_ids, masked_pos)
# logits_lm = logits_lm.data.max(2)[1][0].data.cpu().numpy()
# print('masked tokens list : ',[pos.item() for pos in masked_tokens[0] if pos.item() != 0])
# print('predict masked tokens list : ',[pos for pos in logits_lm if pos != 0])

# logits_clsf = logits_clsf.data.max(1)[1].data.cpu().numpy()[0]
# print('isNext : ', True if isNext else False)
# print('predict isNext : ',True if logits_clsf else False)